In [2]:
%run 1-workspace_setup.ipynb

In [3]:
dem_norm = read_main('county_stats_normalized')
dem_cnt = read_main('county_stats_counts')

edu_norm = read_main('education_county_rates')
edu_cnt = read_main('education_county_counts')

cr_base_rate = read_main('crime_agg_base_rate')
cr_base_count = read_main('crime_agg_base_count')
cr_cat = read_main('crime_agg_category')
cr_name = read_main('crime_agg_name')
cr_cat_rate = read_main('crime_agg_category_rate')
cr_name_rate = read_main('crime_agg_name_rate')

head(dem_norm, dem_cnt, edu_norm, edu_cnt, cr_base_count, cr_base_rate, cr_cat, cr_name, cr_cat_rate, cr_name_rate)

58 cols x 512 rows


,year,county,age_over18_prop,age_undr19_prop,gend_m_prop,gend_f_prop,age_undr19_gend_m_prop,age_undr19_gend_f_prop,age_over18_gend_m_prop,age_over18_gend_f_prop,...,hu_per_cap,age_median,inc_hh_median,hh_size_avg,hu_oo_freq_val_ord,hu_oo_freq_val,hu_blt_freq_yr_ord,hu_blt_freq_yr,med_hm_val,med_yr_blt
0,2012,ADAMS,0.700040,0.299960,2.104797,0.872130,2.341611,0.953744,2.003325,0.837159,...,0.704946,32.4,56633.0,2.91,3,v150k_250k,7,b2000_2009,188100.0,1983.0
1,2012,ALAMOSA,0.718650,0.281350,2.187780,0.803145,2.611540,0.928669,2.021878,0.754002,...,0.840087,32.2,38045.0,2.49,2,v50k_150k,1,b1949_e,133200.0,1974.0
2,2012,ARAPAHOE,0.725375,0.274625,2.279007,0.797066,2.828618,0.951183,2.070926,0.738717,...,0.814089,35.7,60400.0,2.55,3,v150k_250k,4,b1970_1979,230900.0,1982.0


51 cols x 512 rows


,year,county,pop,gend_m,gend_f,age_over18,age_undr19,gend_m_age_undr19,gend_f_age_undr19,gend_m_age_over18,...,hu_blt_1990_1999,hu_blt_1980_1989,hu_blt_1970_1979,hu_blt_1960_1969,hu_blt_1950_1959,hu_blt_lt_1950,ps_known,ps_below,ps_undr18_known,ps_undr18_below
0,2012,ADAMS,231571.0,487410.0,201960.0,162109.0,69462.0,162653.0,66249.0,324757.0,...,27598.0,20368.0,30185.0,19615.0,20369.0,6158.0,438171.0,62008.0,124375.0,25278.0
1,2012,ALAMOSA,7823.0,17115.0,6283.0,5622.0,2201.0,5748.0,2044.0,11367.0,...,866.0,862.0,1405.0,654.0,591.0,1536.0,14622.0,3191.0,3817.0,758.0
2,2012,ARAPAHOE,292548.0,666719.0,233180.0,212207.0,80341.0,227254.0,76419.0,439465.0,...,33989.0,56011.0,62253.0,22258.0,16519.0,7165.0,568999.0,66945.0,144576.0,23054.0


60 cols x 63 rows


...

,year,county,arson,assault_aggravated,assault_simple,bribery,burglary,credit_card_machine_fraud,drug_equipment,drug_narcotic,...,shoplifting,sodomy,stolen_property,theft_from_building,theft_from_vehicle,theft_from_vending_machine,vehicle_part_theft,vehicle_theft,weapon_law,wire_fraud
0,1997,ADAMS,86,602,1954,2,1931,44,725,1120,...,1323,3,245,1232,1733,53,1293,1317,274,0
1,1997,ALAMOSA,1,14,72,0,13,0,8,27,...,0,0,0,14,7,10,1,6,4,1
2,1997,ARAPAHOE,138,671,2956,0,3200,194,916,2598,...,4134,0,421,2283,3867,72,677,2537,695,2


22 cols x 1397 rows


,year,county,arson_rate,assault_rate,bribery_rate,burglary_rate,drug_rate,embezzlement_rate,extortion_rate,forgery_rate,...,kidnapping_rate,larceny_theft_rate,porn_rate,property_damage_rate,prostitution_rate,robbery_rate,sex_offense_rate,stolen_property_rate,vehicle_theft_rate,weapon_law_rate
0,1997,ADAMS,51.269822,1592.941457,1.192321,1151.186360,1099.916537,15.500179,2.980804,145.463217,...,29.808036,4782.997496,0.596161,3259.210683,8.346250,112.674377,188.386789,146.059378,785.143675,163.348039
1,1997,ALAMOSA,13.941168,1282.587481,0.000000,181.235188,487.940889,0.000000,0.000000,41.823505,...,0.000000,2300.292765,0.000000,1017.705284,0.000000,0.000000,125.470514,0.000000,83.647010,55.764673
2,1997,ARAPAHOE,61.190827,1716.890442,0.000000,1418.917721,1558.149022,84.691651,5.320941,297.972721,...,110.409535,6360.742094,0.000000,2596.619429,87.795534,105.532005,192.440716,186.676363,1124.935705,308.171192


44 cols x 1397 rows


,year,county,arson_rate,assault_aggravated_rate,assault_simple_rate,bribery_rate,burglary_rate,credit_card_machine_fraud_rate,drug_equipment_rate,drug_narcotic_rate,...,shoplifting_rate,sodomy_rate,stolen_property_rate,theft_from_building_rate,theft_from_vehicle_rate,theft_from_vending_machine_rate,vehicle_part_theft_rate,vehicle_theft_rate,weapon_law_rate,wire_fraud_rate
0,1997,ADAMS,51.269822,358.888756,1164.898057,1.192321,1151.186360,26.231072,432.216526,667.700012,...,788.720639,1.788482,146.059378,734.470013,1033.146536,31.596518,770.835817,785.143675,163.348039,0.000000
1,1997,ALAMOSA,13.941168,195.176356,1003.764115,0.000000,181.235188,0.000000,111.529346,376.411543,...,0.000000,0.000000,0.000000,195.176356,97.588178,139.411683,13.941168,83.647010,55.764673,13.941168
2,1997,ARAPAHOE,61.190827,297.529310,1310.725244,0.000000,1418.917721,86.021887,406.165197,1151.983824,...,1833.064330,0.000000,186.676363,1012.309111,1714.673383,31.925649,300.189780,1124.935705,308.171192,0.886824


In [4]:
INDEX = ['year', 'county']

def non_idx_cols(df, index=INDEX):
    return [col for col in df.columns if col not in index]

In [5]:
cr_name = cr_name.prefix_cols(non_idx_cols(cr_name), 'CRIME_NAME_COUNT__')
cr_cat = cr_cat.prefix_cols(non_idx_cols(cr_cat), 'CRIME_CATEGORY_COUNT__')

cr_name_rate = cr_name_rate.prefix_cols(non_idx_cols(cr_name_rate), 'CRIME_NAME_NORM__')
cr_cat_rate = cr_cat_rate.prefix_cols(non_idx_cols(cr_cat_rate), 'CRIME_CATEGORY_NORM__')

cr_base_count = cr_base_count.prefix_cols(non_idx_cols(cr_base_count), 'CRIME_BASE_COUNT__')
cr_base_rate = cr_base_rate.prefix_cols(non_idx_cols(cr_base_rate), 'CRIME_BASE_NORM__')

dem_cnt = dem_cnt.prefix_cols(non_idx_cols(dem_cnt), 'CENSUS_COUNT__')
dem_norm = dem_norm.prefix_cols(non_idx_cols(dem_norm), 'CENSUS_NORM__')

edu_cnt = edu_cnt.prefix_cols(non_idx_cols(edu_cnt, ['county']), 'EDU_COUNT__')
edu_norm = edu_norm.prefix_cols(non_idx_cols(edu_norm, ['county']), 'EDU_NORM__')

head(cr_name, cr_cat, cr_name_rate, cr_cat_rate, cr_base_count, cr_base_rate, dem_cnt, dem_norm, edu_cnt, edu_norm)

44 cols x 1397 rows


,year,county,CRIME_NAME_COUNT__arson,CRIME_NAME_COUNT__assault_aggravated,CRIME_NAME_COUNT__assault_simple,CRIME_NAME_COUNT__bribery,CRIME_NAME_COUNT__burglary,CRIME_NAME_COUNT__credit_card_machine_fraud,CRIME_NAME_COUNT__drug_equipment,CRIME_NAME_COUNT__drug_narcotic,...,CRIME_NAME_COUNT__shoplifting,CRIME_NAME_COUNT__sodomy,CRIME_NAME_COUNT__stolen_property,CRIME_NAME_COUNT__theft_from_building,CRIME_NAME_COUNT__theft_from_vehicle,CRIME_NAME_COUNT__theft_from_vending_machine,CRIME_NAME_COUNT__vehicle_part_theft,CRIME_NAME_COUNT__vehicle_theft,CRIME_NAME_COUNT__weapon_law,CRIME_NAME_COUNT__wire_fraud
0,1997,ADAMS,86,602,1954,2,1931,44,725,1120,...,1323,3,245,1232,1733,53,1293,1317,274,0
1,1997,ALAMOSA,1,14,72,0,13,0,8,27,...,0,0,0,14,7,10,1,6,4,1
2,1997,ARAPAHOE,138,671,2956,0,3200,194,916,2598,...,4134,0,421,2283,3867,72,677,2537,695,2


22 cols x 1397 rows


,year,county,CRIME_CATEGORY_COUNT__arson,CRIME_CATEGORY_COUNT__assault,CRIME_CATEGORY_COUNT__bribery,CRIME_CATEGORY_COUNT__burglary,CRIME_CATEGORY_COUNT__drug,CRIME_CATEGORY_COUNT__embezzlement,CRIME_CATEGORY_COUNT__extortion,CRIME_CATEGORY_COUNT__forgery,...,CRIME_CATEGORY_COUNT__kidnapping,CRIME_CATEGORY_COUNT__larceny_theft,CRIME_CATEGORY_COUNT__porn,CRIME_CATEGORY_COUNT__property_damage,CRIME_CATEGORY_COUNT__prostitution,CRIME_CATEGORY_COUNT__robbery,CRIME_CATEGORY_COUNT__sex_offense,CRIME_CATEGORY_COUNT__stolen_property,CRIME_CATEGORY_COUNT__vehicle_theft,CRIME_CATEGORY_COUNT__weapon_law
0,1997,ADAMS,86,2672,2,1931,1845,26,5,244,...,50,8023,1,5467,14,189,316,245,1317,274
1,1997,ALAMOSA,1,92,0,13,35,0,0,3,...,0,165,0,73,0,0,9,0,6,4
2,1997,ARAPAHOE,138,3872,0,3200,3514,191,12,672,...,249,14345,0,5856,198,238,434,421,2537,695


44 cols x 1397 rows


...

,year,county,CENSUS_NORM__age_over18_prop,CENSUS_NORM__age_undr19_prop,CENSUS_NORM__gend_m_prop,CENSUS_NORM__gend_f_prop,CENSUS_NORM__age_undr19_gend_m_prop,CENSUS_NORM__age_undr19_gend_f_prop,CENSUS_NORM__age_over18_gend_m_prop,CENSUS_NORM__age_over18_gend_f_prop,...,CENSUS_NORM__hu_per_cap,CENSUS_NORM__age_median,CENSUS_NORM__inc_hh_median,CENSUS_NORM__hh_size_avg,CENSUS_NORM__hu_oo_freq_val_ord,CENSUS_NORM__hu_oo_freq_val,CENSUS_NORM__hu_blt_freq_yr_ord,CENSUS_NORM__hu_blt_freq_yr,CENSUS_NORM__med_hm_val,CENSUS_NORM__med_yr_blt
0,2012,ADAMS,0.700040,0.299960,2.104797,0.872130,2.341611,0.953744,2.003325,0.837159,...,0.704946,32.4,56633.0,2.91,3,v150k_250k,7,b2000_2009,188100.0,1983.0
1,2012,ALAMOSA,0.718650,0.281350,2.187780,0.803145,2.611540,0.928669,2.021878,0.754002,...,0.840087,32.2,38045.0,2.49,2,v50k_150k,1,b1949_e,133200.0,1974.0
2,2012,ARAPAHOE,0.725375,0.274625,2.279007,0.797066,2.828618,0.951183,2.070926,0.738717,...,0.814089,35.7,60400.0,2.55,3,v150k_250k,4,b1970_1979,230900.0,1982.0


78 cols x 63 rows


,county,EDU_COUNT__pupil_total,EDU_COUNT__stable,EDU_COUNT__mobile,EDU_COUNT__mobile_instances,EDU_COUNT__disabled_pupil_total,EDU_COUNT__disabled_stable,EDU_COUNT__disabled_mobile,EDU_COUNT__disabled_mobile_instances,EDU_COUNT__limited_eng_pupil_total,...,EDU_COUNT__migrant_completed,EDU_COUNT__title_1_grad_base_total,EDU_COUNT__title_1_graduated,EDU_COUNT__title_1_completed,EDU_COUNT__homeless_grad_base_total,EDU_COUNT__homeless_graduated,EDU_COUNT__homeless_completed,EDU_COUNT__gifted_grad_base_total,EDU_COUNT__gifted_graduated,EDU_COUNT__gifted_completed
0,ADAMS,98546,67272,31222,33925,8848,6263,2588,2896,20773,...,33,935,529,559,360,190,204,402,337,345
1,ALAMOSA,2775,1882,885,950,223,159,63,66,368,...,4,28,22,23,6,6,6,0,0,0
2,ARAPAHOE,124639,94109,30134,32269,11842,9461,2354,2568,25370,...,9,488,202,213,243,96,102,909,820,828


60 cols x 63 rows


,county,EDU_NORM__stable_rate,EDU_NORM__mobile_rate,EDU_NORM__mobile_instances_rate,EDU_NORM__disabled_stable_rate,EDU_NORM__disabled_mobile_rate,EDU_NORM__disabled_mobile_instances_rate,EDU_NORM__disabled_graduated_rate,EDU_NORM__disabled_completed_rate,EDU_NORM__limited_eng_stable_rate,...,EDU_NORM__white_mobile_instances_rate,EDU_NORM__black_stable_rate,EDU_NORM__black_mobile_rate,EDU_NORM__black_mobile_instances_rate,EDU_NORM__hispanic_stable_rate,EDU_NORM__hispanic_mobile_rate,EDU_NORM__hispanic_mobile_instances_rate,EDU_NORM__asian_stable_rate,EDU_NORM__asian_mobile_rate,EDU_NORM__asian_mobile_instances_rate
0,ADAMS,68.26,31.68,34.43,70.78,29.25,32.73,47.54,50.10,69.99,...,32.45,54.66,45.08,47.91,67.49,32.71,36.40,78.55,21.37,23.54
1,ALAMOSA,67.82,31.89,34.23,71.30,28.25,29.60,86.67,93.33,72.01,...,35.54,57.14,42.86,42.86,70.15,29.47,32.69,64.00,36.00,36.00
2,ARAPAHOE,75.51,24.18,25.89,79.89,19.88,21.69,51.26,52.06,74.94,...,21.16,67.47,32.03,34.57,72.76,26.75,29.20,78.59,21.30,22.56


In [6]:
df = (
    # CRIME
    cr_name
    .merge(cr_cat, on=INDEX)
    .merge(
        (
            cr_name_rate
            .merge(cr_cat_rate, on=INDEX)
        ),
        on=INDEX
    )
    .merge(
        (
            cr_base_count
            .merge(cr_base_rate, on=INDEX)
        ),
        on=INDEX
    )
    # DEMOGRAPHICS
    .merge(
        (
            dem_cnt
            .merge(dem_norm, on=INDEX)
        ),
        on=INDEX
    )
    # EDUCATION
    .merge(
        (
            edu_cnt
            .merge(edu_norm, on=['county'])
        ),
        on=['county']
    )
)

In [7]:
# Change all numbers to floats, and round
numeric_cols = df.select_dtypes(include=[np.number])
df[numeric_cols.columns] = numeric_cols.astype(float).round(2)
df.year = df.year.astype(int)
head(df)

381 cols x 490 rows


,year,county,CRIME_NAME_COUNT__arson,CRIME_NAME_COUNT__assault_aggravated,CRIME_NAME_COUNT__assault_simple,CRIME_NAME_COUNT__bribery,CRIME_NAME_COUNT__burglary,CRIME_NAME_COUNT__credit_card_machine_fraud,CRIME_NAME_COUNT__drug_equipment,CRIME_NAME_COUNT__drug_narcotic,...,EDU_NORM__white_mobile_instances_rate,EDU_NORM__black_stable_rate,EDU_NORM__black_mobile_rate,EDU_NORM__black_mobile_instances_rate,EDU_NORM__hispanic_stable_rate,EDU_NORM__hispanic_mobile_rate,EDU_NORM__hispanic_mobile_instances_rate,EDU_NORM__asian_stable_rate,EDU_NORM__asian_mobile_rate,EDU_NORM__asian_mobile_instances_rate
0,2012,ADAMS,105.0,1029.0,2835.0,2.0,2311.0,535.0,3211.0,3522.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,73.0,899.0,2649.0,7.0,2144.0,437.0,1965.0,2204.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,123.0,1033.0,2863.0,8.0,2073.0,526.0,1945.0,2145.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54


In [8]:
## TEMPORARY - REVERSE OUR WORK
df.columns = [c if '__' not in c else c.split('__')[1] for c in df.columns]
df = df.loc[:, ~ df.columns.duplicated()]
head(df)

353 cols x 490 rows


,year,county,arson,assault_aggravated,assault_simple,bribery,burglary,credit_card_machine_fraud,drug_equipment,drug_narcotic,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,2012,ADAMS,105.0,1029.0,2835.0,2.0,2311.0,535.0,3211.0,3522.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,73.0,899.0,2649.0,7.0,2144.0,437.0,1965.0,2204.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,123.0,1033.0,2863.0,8.0,2073.0,526.0,1945.0,2145.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54


In [9]:
write_main(df, 'county_EVERYTHING')

490

## District

In [10]:
df_dist = read_main('education_dist')
df_geo_dist = read_main('''
    d.district,
    c.county,
    d.geo_border AS geo_district_border,
    c.geo_border AS geo_county_border,
    d.geo_point AS geo_district_point,
    c.geo_point AS geo_county_point
FROM district AS d
INNER JOIN county AS c
    ON c.county = d.in_county
''')

In [11]:
df_dist = (df_geo_dist
    .merge(df_dist, on=['district'])
    [ ~ df_geo_dist.geo_district_border.isna()]
)

head(df_dist)

142 cols x 178 rows


,district,county,geo_district_border,geo_county_border,geo_district_point,geo_county_point,pupil_total,stable,mobile,mobile_instances,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,MAPLETON 1,ADAMS,MULTIPOLYGON (((-105.01581612299998 39.8144774...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,9037,5077,3919,4133,...,52.08,48.04,51.40,53.63,60.31,39.19,42.19,47.22,52.78,53.70
1,ADAMSFIVESTAR 12,ADAMS,MULTIPOLYGON (((-105.05310614499996 39.9302934...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,49889,34283,15424,16854,...,32.27,55.98,43.94,47.41,67.30,32.23,36.81,81.07,18.84,21.15
2,ADAMSCOUNTY 14,ADAMS,MULTIPOLYGON (((-104.96883410999999 39.7910064...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,8265,5510,3038,3397,...,47.45,48.20,51.80,53.60,68.67,35.20,39.57,80.00,20.00,20.00


In [12]:
write_main(df_dist, 'district_EVERYTHING')

178

## Generate combined notebook output

In [14]:
from combine_notebooks import combine

combine(
    filename="8-combined-AUTO-GENERATED-FILE",
    line_filters = "%run 1-workspace_setup.ipynb"
)